In [17]:
import re,string,nltk,csv
import gensim
import pandas as pd
from textblob import TextBlob
from gensim import corpora
from nltk.tokenize import word_tokenize , TweetTokenizer
from nltk.stem import WordNetLemmatizer
from gensim.models import LdaModel, LdaMulticore
from nltk.corpus import stopwords
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
import datetime


    
data = pd.read_csv("/Users/keshavgaddhyan/Desktop/NLP project/trump_tweets.csv")
tknzr = TweetTokenizer()
wnl=WordNetLemmatizer()
stop_words = stopwords.words('english') + ['...']
punctuation=set(string.punctuation)
data["text"]

0        RT @GOPChairwoman: House Democrats are engaged...
1        RT @rww_gop: Voters are rejecting the Dems’ ba...
2        As the Witch Hunt continues! https://t.co/klvv...
3        The Greatest Witch Hunt in American History! h...
4        RT @realDonaldTrump: Years of BAD GOVERNMENT, ...
                               ...                        
11407    The forgotten men and women of our country wil...
11408    January 20th 2017, will be remembered as the d...
11409    What truly matters is not which party controls...
11410    power from Washington, D.C. and giving it back...
11411    Today we are not merely transferring power fro...
Name: text, Length: 11412, dtype: object

In [18]:
def clean_tweet(tweet):
        tweet=tweet.lower()          # converts the tweets tro lower case
        tweet=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet) # removes links in the tweets
        tweet=tknzr.tokenize(tweet)              #tokenize the tweets into words
        new_tweet=[]
        for word in tweet:
            word=wnl.lemmatize(word) #lemmatize the words
            if word not in (stop_words) and word not in punctuation: # remove stop words and punctuations from the tweets
                new_tweet.append(word)
        return new_tweet
for row in data.iterrows():
    row[1]["text"]=clean_tweet(row[1]["text"])
data.head()

,text,created_at,Unnamed: 2
0,"[rt, @gopchairwoman, house, democrat, engaged,...",10-17-2019 15:09:09,NaN
1,"[rt, @rww_gop, voter, rejecting, dems, ’, base...",10-17-2019 15:08:35,NaN
2,"[witch, hunt, continues]",10-17-2019 15:07:46,NaN
3,"[greatest, witch, hunt, american, history]",10-17-2019 13:14:49,NaN
4,"[rt, @realdonaldtrump, year, bad, government, ...",10-17-2019 12:59:25,NaN


In [19]:

dictionary = corpora.Dictionary(data["text"])
# print(dictionary)
# mycorpus is a list of list|
# where each list is a tweet and it is a list of tuples [(token_id, no_of_times_it_occurs_in_tweet),(),...]
mycorpus = [dictionary.doc2bow(tweet, allow_update=True) for tweet in data["text"]]

word_counts = [[(dictionary[id], count) for id, count in line] for line in mycorpus]

# lda_model = LdaModel(corpus=mycorpus,id2word=dictionary, num_topics=30,passes=10)
lda_model = gensim.models.ldamodel.LdaModel(corpus=mycorpus,id2word=dictionary,
num_topics=30, passes = 10)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))



Topic: 0 
Words: 0.044*"bill" + 0.033*"hurricane" + 0.031*"care" + 0.028*"yesterday" + 0.028*"c" + 0.020*"signed" + 0.019*"thanks" + 0.019*"local" + 0.018*"sign" + 0.018*"support"
Topic: 1 
Words: 0.059*"looking" + 0.041*"@flotus" + 0.036*"god" + 0.034*"friend" + 0.033*"south" + 0.032*"wonderful" + 0.027*"action" + 0.026*"carolina" + 0.025*"japan" + 0.024*"truly"
Topic: 2 
Words: 0.046*"u" + 0.032*"deal" + 0.031*"year" + 0.025*"trade" + 0.024*"china" + 0.021*"ha" + 0.021*"many" + 0.021*"country" + 0.019*"billion" + 0.019*"dollar"
Topic: 3 
Words: 0.111*"thank" + 0.055*"working" + 0.053*"hard" + 0.042*"great" + 0.031*"night" + 0.027*"last" + 0.026*"big" + 0.020*"together" + 0.019*"rally" + 0.018*"see"
Topic: 4 
Words: 0.043*"job" + 0.042*"ever" + 0.035*"history" + 0.033*"economy" + 0.028*"number" + 0.027*"since" + 0.027*"year" + 0.027*"best" + 0.026*"record" + 0.024*"market"
Topic: 5 
Words: 0.048*"watch" + 0.031*"build" + 0.025*"interview" + 0.025*"confidence" + 0.023*"leadership" + 0.

In [ ]:
trade_tweets=["trade", "currency", "economy", "growth",  "trade-war", "rates", "inflation", "manipulation", "dollar", "Fed", "Powell", "tariffs", ]
for i in range(len(trade_tweets)):
    trade_tweets[i]=wnl.lemmatize(trade_tweets[i])  #lemmatize the words in our bag of words for trade realted tweets as well so that it is coherant with our lemmatized tweeets
def istrade_tweet(tweet):
    for word in trade_tweets: 
        if word in tweet:
            return True
        
    return False
        
        
            
for index,row in data.iterrows():
    if (istrade_tweet(row["text"]))==False:
        data.drop(index, inplace=True)
        
data

In [ ]:
data.insert(1,"Sentiment","Positive")
def analyze_sentiment(tweet):
    listToStr = ' '.join(map(str, tweet))
    analysis = TextBlob(listToStr)
    if analysis.sentiment.polarity > 0.7:
        return 'Positive'
    elif analysis.sentiment.polarity < -0.7:
        return 'Negative'
    else:
        return 'Neutral'

for index,row in data.iterrows():
    row["Sentiment"]=analyze_sentiment(row["text"])
    
data
# data.to_csv(r'/Users/keshavgaddhyan/Desktop/NLP project/sentiment.csv')
        
        

In [ ]:
sp500=pd.read_csv("/Users/keshavgaddhyan/Desktop/NLP project/yahoo finance s&p500.csv")
# sp500.drop(["High", "Low","Adj Close","Volume"], axis = 0) 

# x axis values 
# x = [1,2,3] 
# # corresponding y axis values 
# y = [2,4,1] 
figure(num=None, figsize=(15, 15), dpi=80, facecolor='w', edgecolor='k')
sp500['Date'] = pd.to_datetime(sp500['Date'])
data['created_at'] = pd.to_datetime(data['created_at'])
sp500.set_index('Date')['Close'].plot();


# plotting the points
# plt.plot(sp500["Date"], sp500["Close"])
# plt.show()

In [ ]:
for index,row in data.iterrows():
    if row["Sentiment"]=="Neutral":
        data.drop(index, inplace=True)

sp500.insert(1,"Portfolio",0)
sp500.insert(1,"Units",0)
sp500.insert(1,"Cash",0)
sp500.loc[0,"Units"]=1
sp500.loc[0,"Portfolio"]=2269.959961

data


        

In [ ]:


# for index,row in sp500.iterrows():
#     print(row["Date"])
prevunit=1
prevcash=0
prevport=0
action="none"
opening_time=datetime.time(9,30,0)
closing_time=datetime.time(16,30,0)
for index,row in sp500.iterrows():
    count=0
    for idx,rows in data.iterrows():  
        if (row["Date"].date())==(rows["created_at"].date()):
            
            if rows["created_at"].time() > opening_time and rows["created_at"].time() < closing_time:
                print(rows["text"])
                if rows["Sentiment"]=="Positive":
                    units_bought=2000/row["Close"]  #borrowed 2000 at risk free rate
                    prevunit=prevunit+units_bought
                    sp500.loc[index,"Units"]=prevunit
                    sp500.loc[index,"Portfolio"]=prevunit*row["Close"]
                    sp500.loc[index,"Cash"]=prevcash
                    action="bought_by_borrowing"
                    count=1
                    
                else:
                    #we are shorting the stock and also selling what we own
#                     row["Cash"]+=row["Units"]*row["Close"]
#                     row["Cash"]+=2000
                    # shorting for money worth 2000 so adding it temorsrily to cash and then subtract tomorrow

                    prevcash=prevcash+(prevunit*row["Close"])+2000
                    sp500.loc[index,"Cash"]=prevcash
                    prevunit=0
                    sp500.loc[index,"Units"]= prevunit
                    action="Shorted"
                    sp500.loc[index,"Portfolio"]=0
                    prevunit=-2000/row["Close"]
                    sp500.loc[index,"Units"]=prevunit       
                    count=1
                    
                    
                
            if rows["created_at"].time() < opening_time: 
                print(rows["text"],rows["Sentiment"])
                if rows["Sentiment"]=="Positive":   # we borrowed money in the morning and returned at market close, assuming no interest rate for this
                    units_bought=2000/row["Open"]
                    money_received=units_bought*row["Close"]
                    prevcash+=money_received-2000
                    sp500.loc[index,"Units"]=prevunit
                    sp500.loc[index,"Portfolio"]=prevunit*row["Close"]
                    sp500.loc[index,"Cash"]=prevcash
                    count=1

                else:
                    prevcash+=(prevunit*(row["Open"]-row["Close"])) + 2000
                    prevcash=prevcash-(2000/row["Open"])*row["Close"] # the shorted amount
                    sp500.loc[index,"Units"]=prevunit
                    sp500.loc[index,"Portfolio"]=prevunit*row["Close"]
                    sp500.loc[index,"Cash"]=prevcash 
                    count=1
                    
            if rows["created_at"].time() > closing_time :
                print(rows["text"],rows["created_at"],rows["Sentiment"])
                if rows["Sentiment"]=="Positive":
                    action="after_hours_positive"
                    count=1
                else:
                    action="after_hours_negative"
                    count=1    
                
# rows["created_at"].hour >16
              
   

    if count==0:
        
        if action=="bought_by_borrowing":
#             row["Cash"]=(row["Units"]-1)*row["Close"]
#             row["Cash"]-=2000+(2000*(1.78/100)*(1/365))  # returning the 2k borrowed plus the interest at risk free rate of 1.78 percent
#             sp500.loc[index,"Units"]=1
#             sp500.loc[index,"Portfolio"]=row["Units"]*row["Close"]
            action="none"
            prevcash+=((prevunit-1)*row["Close"])-(2000+(2000*(1.78/100)*(1/365)))  # returning the 2k borrowed plus the interest at risk free rate of 1.78 percent
            #we sold everything except 1 unit and then returned the borrowed sum
            prevunit=1
        elif action=="Shorted":
            
#             row["Cash"]-=(-row["Units"]+1)*row["Close"] #subtracted the shorted units from my cash
#             sp500.loc[index,"Units"]=1
#             sp500.loc[index,"Portfolio"]=row["Units"]*row["Close"]
            action="none"
            prevcash=prevcash-((1-prevunit)*row["Close"]) #subtracted the shorted units from my cash
            prevunit=1
            
        elif action=="after_hours_positive":
            units_bought=2000/row["Open"]
            money_received=units_bought*row["Close"]
            print(money_received)
            prevcash+=money_received-2000
            action="none"
            prevunit=1
#             sp500.loc[index,"Units"]=prevunit
#             sp500.loc[index,"Portfolio"]=prevunit*row["Close"]
#             sp500.loc[index,"Cash"]=prevcash
            
        elif action == "after_hours_negative":
            prevcash+=(prevunit*(row["Open"]-row["Close"])) + 2000
            prevcash=prevcash-((2000/row["Open"])*row["Close"]) # the shorted amount
            action="none"
            prevunit=1
#             sp500.loc[index,"Units"]=prevunit
#             sp500.loc[index,"Portfolio"]=prevunit*row["Close"]
#             sp500.loc[index,"Cash"]=prevcash 
            
            
            
               
        sp500.loc[index,"Units"]=prevunit
        sp500.loc[index,"Portfolio"]=row["Units"]*row["Close"]
        sp500.loc[index,"Cash"]=prevcash

            
            
            
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(sp500)
        
        
        
        
        

# for index,row in data.iterrows():
#     print(row["created_at"].time())

